In [189]:
"""Backend supported: tensorflow.compat.v1, tensorflow, pytorch, paddle"""

# Import tf if using backend tensorflow.compat.v1 or tensorflow
from deepxde.backend import tf
import deepxde as dde
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import plotly.graph_objects as go
import sympy as sp
import scipy.integrate as spi
from scipy.interpolate import interp1d
import os
import imageio



def generate_evolution_model(domain, boundary, width, depth, rate, epochs):
    
    
    seed = 42

    dde.config.set_random_seed(seed)
    
    n_history = 10
    
    
    def pde(x, y):
        # y = (f)
        # x = (x,y,z)
        # x = x[:,0:1]
        # y = x[:,1:2]
        # z = x[:,2:]
        df_x = dde.grad.jacobian(y, x, i=0, j=0)
        df_y = dde.grad.jacobian(y, x, i=0, j=1)
        df_z = dde.grad.jacobian(y, x, i=0, j=2)
        first_factor = (x[:,0:1]**2 + x[:,1:2]**2)/(x[:,0:1]**2 + x[:,1:2]**2 + x[:,2:3]**2)
        squared_terms = ((x[:,0:1]**2)*(df_x)) + ((x[:,1:2]**2)*(df_y)) + ((x[:,2:]**2)*(df_z))
        cross_terms =  (x[:,0:1] * x[:,1:2] * df_x * df_y)+(x[:,0:1] * x[:,2:] * df_x * df_z)+(x[:,1:2] * x[:,2:] * df_y * df_z)
        return first_factor * (squared_terms + (2*cross_terms)) - 0.5

    

    geom = dde.geometry.geometry_3d.Cuboid([-100,-100,-100],[100,100,100])
   
    # singularity = dde.geometry.geometry_3d.Sphere([0,0,0], 0.1)



    icx_1 = dde.icbc.DirichletBC(
        geom,
        lambda x: 1,
        lambda x, on_boundary:  np.isclose(x[0],50),
    )
    
    
    
    icx_2 = dde.icbc.DirichletBC(
        geom,
        lambda x: 0,
        lambda x, on_boundary:  np.isclose(x[0],100),
    )
    
    
    
    
    icy_1 = dde.icbc.DirichletBC(
        geom,
        lambda x: 1,
        lambda x, on_boundary:  np.isclose(x[1],60),
    )
    
    icy_2 = dde.icbc.DirichletBC(
        geom,
        lambda x: 0,
        lambda x, on_boundary:  np.isclose(x[1],100),
    )
    
    
    
    
    
    
    icz_1 = dde.icbc.DirichletBC(
        geom,
        lambda x: 1,
        lambda x, on_boundary:  np.isclose(x[2],60),
    )
    
    icz_2 = dde.icbc.DirichletBC(
        geom,
        lambda x: 0,
        lambda x, on_boundary:  np.isclose(x[2],100),
    )
    
    
    
    

    data = dde.data.PDE(
        geom, pde, [icx_1, icx_2, icy_1, icy_2, icz_1, icz_2], num_domain=domain, num_boundary=boundary)


    net = dde.nn.FNN([3] + [width] * depth + [1], "tanh", "Glorot normal")


  
    
    
    checker = dde.callbacks.ModelCheckpoint(
    "temp_model_history/model", save_better_only=False, period= (epochs/100))
    model = dde.Model(data, net)
    model.compile("adam", lr=rate)
    model.train(iterations=epochs, callbacks = [checker])
    
    
    
    print("Training Complete!")

    return model




In [190]:
model = generate_evolution_model(200, 200, 10, 1, 0.01, 10000)

Compiling model...
Building feed-forward neural network...
'build' took 0.065711 s

'compile' took 1.738152 s

Training model...

Step      Train loss                                                                Test loss                                                                 Test metric
0         [4.09e+04, 1.21e+01, 1.17e+00, 6.53e+00, 1.90e+00, 8.32e-01, 5.80e+00]    [4.09e+04, 1.21e+01, 1.17e+00, 6.53e+00, 1.90e+00, 8.32e-01, 5.80e+00]    []  
INFO:tensorflow:temp_model_history/model-100.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:temp_model_history/model-200.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:temp_model_history/model-300.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:temp_model_history/model-400.ckpt is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:temp_model_history/model-500.ckpt is not in all_model_checkpoint_paths. Manually adding i

In [191]:
model_history = []

epochs = 10000
n_history = 1000

for i in range (1,101):
    model.restore(f"temp_model_history/model-{((i)*100)}.ckpt", verbose=1)
    model_history.append(model)  # Replace ? with the exact filename

Restoring model from temp_model_history/model-100.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-100.ckpt
Restoring model from temp_model_history/model-200.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-200.ckpt
Restoring model from temp_model_history/model-300.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-300.ckpt
Restoring model from temp_model_history/model-400.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-400.ckpt
Restoring model from temp_model_history/model-500.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-500.ckpt
Restoring model from temp_model_history/model-600.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-600.ckpt
Restoring model from temp_model_history/model-700.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-700.ckpt
Restoring model from temp_model_history/model-80

In [196]:
import imageio
import io

# Generate sample data points
x_values = np.linspace(-10, 10, 30)
y_values = np.linspace(-10, 10, 30)
z_values = np.linspace(-10, 10, 30)
X, Y, Z = np.meshgrid(x_values, y_values, z_values)


frames = []
frames_xc = []
frames_yc = []
frames_zc = []
frames_xs = []
frames_ys = []
frames_zs = []


for i in range(len(model_history)):
    # Evaluate the model on the data points
    model.restore(f"temp_model_history/model-{((i+1)*100)}.ckpt", verbose=1)
    
    input_data = np.vstack((X.ravel(), Y.ravel(), Z.ravel())).T
    predictions = model.predict(input_data)
    predictions = predictions.reshape(X.shape)
    

    fig = go.Figure(data=[
        go.Scatter3d(x=X.ravel(), y=Y.ravel(), z=Z.ravel(), mode='markers',
                     marker=dict(size=3, colorscale='Inferno',
                                 color=predictions.ravel(),
                                 colorbar=dict(title='Normalized f(x,y,z) value'),
                                 cmin=np.min(predictions), cmax=np.max(predictions))),
    ])

    fig.update_layout(scene=dict(
                        xaxis_title='X',
                        yaxis_title='Y',
                        zaxis_title='Z'),
                    title=f'Warp Drive Shape Function (Epoch = {(i+1)*100})')

    # Convert Plotly figure to bytes
    img_bytes = fig.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames.append(np.array(img_pil))

 

  
    # Define the slices for x=0, y=0, z=0
    x_slice = 0
    y_slice = 0
    z_slice = 0

    # Plot the 3D surface for x=0 slice
    fig_xc = go.Figure(data=[go.Contour(x=y_values, y=z_values, z=predictions[1, :, :])])
    fig_xc.update_layout(title=f'Contour slice for x=0 (Epoch = {(i+1)*100})')



    # Convert Plotly figure to bytes
    img_bytes = fig_xc.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_xc.append(np.array(img_pil))


    
    
    
    
    
    # Plot the 3D surface for y=0 slice
    fig_yc = go.Figure(data=[go.Contour(x=x_values, y=z_values, z=predictions[:, y_slice, :])])
    fig_yc.update_layout(title=f'Contour slice for y=0 (Epoch = {(i+1)*100})')
   



    # Convert Plotly figure to bytes
    img_bytes = fig_yc.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_yc.append(np.array(img_pil))


    
    
    
    
    
    
    
    # Plot the 3D surface for z=0 slice
    fig_zc = go.Figure(data=[go.Contour(x=x_values, y=y_values, z=predictions[:, :, z_slice])])
    fig_zc.update_layout(title=f'Contour slice for z=0 (Epoch = {(i+1)*100})')
 

    
    
    # Convert Plotly figure to bytes
    img_bytes = fig_zc.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_zc.append(np.array(img_pil))

  
    

    
    
    

    # Plot the 3D surface for x=0 slice
    fig_xs = go.Figure(data=[go.Surface(x=y_values, y=z_values, z=predictions[1, :, :])])
    fig_xs.update_layout(title=f'Surface slice for x=0 (Epoch = {(i+1)*100})')
  


    # Convert Plotly figure to bytes
    img_bytes = fig_xs.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_xs.append(np.array(img_pil))

    # Save the frames as a GIF

    
    


    # Plot the 3D surface for y=0 slice
    fig_ys = go.Figure(data=[go.Surface(x=x_values, y=z_values, z=predictions[:, y_slice, :])])
    fig_ys.update_layout(title=f'Surface slice for y=0 (Epoch = {(i+1)*100})')
  

    # Convert Plotly figure to bytes
    img_bytes = fig_ys.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_ys.append(np.array(img_pil))

    # Save the frames as a GIF

    

    
    
    
    
    

    # Plot the 3D surface for z=0 slice
    fig_zs = go.Figure(data=[go.Surface(x=x_values, y=y_values, z=predictions[:, :, z_slice])])
    fig_zs.update_layout(title=f'Surface slice for z=0 (Epoch = {(i+1)*100})')
 
    
    # Convert Plotly figure to bytes
    img_bytes = fig_zs.to_image(format="png")
    
    # Convert bytes to PIL image
    img_pil = Image.open(io.BytesIO(img_bytes))
    
    # Convert PIL image to numpy array and append to frames list
    frames_zs.append(np.array(img_pil))

    # Save the frames as a GIF
   
    

    
imageio.mimsave('warp_drive_animation.gif', frames, duration=0.1)
imageio.mimsave('xc.gif', frames_xc, duration=0.1)
imageio.mimsave('yc.gif', frames_yc, duration=0.1)
imageio.mimsave('zc.gif', frames_zc, duration=0.1)
imageio.mimsave('xs.gif', frames_xs, duration=0.1)
imageio.mimsave('ys.gif', frames_ys, duration=0.1)
imageio.mimsave('zs.gif', frames_zs, duration=0.1)

# os.rmdir('temp_model_history')
    
 

Restoring model from temp_model_history/model-100.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-100.ckpt
Restoring model from temp_model_history/model-200.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-200.ckpt
Restoring model from temp_model_history/model-300.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-300.ckpt
Restoring model from temp_model_history/model-400.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-400.ckpt
Restoring model from temp_model_history/model-500.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-500.ckpt
Restoring model from temp_model_history/model-600.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-600.ckpt
Restoring model from temp_model_history/model-700.ckpt ...

INFO:tensorflow:Restoring parameters from temp_model_history/model-700.ckpt
Restoring model from temp_model_history/model-80